In [ ]:
!pip install transformers
!pip install huggingface
!pip install sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
data = pd.read_csv('./drive/MyDrive/JEOPARDY_CSV.csv')

In [ ]:
print(data.columns)

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')


In [ ]:
data.shape

(216930, 7)

In [ ]:
data.head()

NameError: ignored

In [ ]:
category_question_answers = pd.concat([data[' Category'],data[' Question'],data[' Answer']], axis=1)
context = data[' Question']
answer = data[' Answer']
print(context.shape, answer.shape)


(216930,) (216930,)


In [ ]:
#attempt with baseline T5
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Downloading:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

In [ ]:
def get_question(answer, context, max_length=64):
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return tokenizer.decode(output[0])

In [ ]:
#subset 100 datapoints, answer + context 
n = 100

generated = []
for i in range(n):
  con, ans = context[i], answer[i]
  generated_q = get_question(ans, con)
  generated.append([con, ans, generated_q])

In [ ]:
import numpy as np


In [ ]:
q_a_generated = pd.DataFrame(np.array(generated), columns=['context', 'answer', 'generated_question'])

In [ ]:
q_a_generated


,context,answer,generated_question
0,"For the last 8 years of his life, Galileo was ...",Copernicus,<pad> question: Whose theory did Galileo suppo...
1,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,<pad> question: Who was a 1912 Olympian?</s>
2,The city of Yuma in this state has a record av...,Arizona,<pad> question: What state has the most sunshi...
3,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's,<pad> question: What company served its billio...
4,"Signer of the Dec. of Indep., framer of the Co...",John Adams,<pad> question: Who was the second President o...
...,...,...,...
95,"Say <a href=""http://www.j-archive.com/media/20...",Oregon,<pad> question: What state was admitted to the...
96,This car company has been in the news for wide...,Toyota,<pad> question: What car company has been in t...
97,"As an adjective, it can mean proper; as a verb...",correct,<pad> question: What does an adjective mean?</s>
98,The wedge is an adaptation of the simple machi...,plane,<pad> question: What is the shape of the wedge...


In [ ]:
print(q_a_generated.shape)

(100, 3)


context               For the last 8 years of his life, Galileo was ...
answer                                                       Copernicus
generated_question    <pad> question: Whose theory did Galileo suppo...
Name: 0, dtype: object